<a href="https://colab.research.google.com/github/ramkumardeepak774/similarity_Search_using_Faiss/blob/main/search_Using_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
!pip install faiss-cpu
import faiss


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 52.3 MB/s eta 0:00:00


In [ ]:
#import kb data
import pandas as pd 
kb_data = pd.read_csv("/content/deepu.csv")

In [ ]:
# message embedding only on 'title_keywords_messages' column
sentences = kb_data['title_keywords_messages'].tolist()

In [ ]:
len(sentences)

312

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [ ]:
sentence_embeddings = model.encode(sentences)

In [ ]:
#dimension of vector
d = sentence_embeddings.shape[1]

In [ ]:
index_euclidDistance= faiss.IndexFlatL2(d) # index where we search on min euclidian distance 
index_innerProduct = faiss.IndexFlatIP(d) #index where we search on maximum inner product similar to cosine similarity

In [ ]:
index_euclidDistance.add(sentence_embeddings)
index_innerProduct.add(sentence_embeddings)

In [ ]:
# save the index to a binary file
faiss.write_index(index_euclidDistance, "index_euclidDistance.bin")
faiss.write_index(index_innerProduct, "index_innerProduct.bin")

In [ ]:
# load the index from the binary file
index2_euclidDistance = faiss.read_index("index_euclidDistance.bin")
index2_innerProduct = faiss.read_index("index_innerProduct.bin")

size of index in memory

In [ ]:
import sys
index2_euclidDistance_sizeInMemory = sys.getsizeof(index2_euclidDistance)
index2_innerProduct_sizeInMemory = sys.getsizeof(index2_innerProduct)
# Print the size of the variable in bytes
print("Size of variable in memory:", index2_euclidDistance_sizeInMemory, "bytes")
print("Size of variable in memory:", index2_innerProduct_sizeInMemory, "bytes")

Size of variable in memory: 48 bytes
Size of variable in memory: 48 bytes


In [ ]:
searchQuery = "For the purpose of administration, the state is divided into 18 divisions and 75 districts. Divisional Commissioner, an IAS officer is the head of administration on the divisional level.[144][146][147] The administration in each district is headed by a District Magistrate, who is also an IAS officer, and is assisted by a number of officers belonging to state services."
searchQueryVector = model.encode(searchQuery)

In [ ]:
t = searchQueryVector.reshape(1,384)
t.shape

(1, 384)

In [ ]:
t.shape

(1, 384)

In [ ]:
# perform a query on the loaded index
D, I = index2_euclidDistance.search(t, k=3)

print(I)

[[ 40 103 192]]


In [ ]:
for index in I[0]:
  print(sentences[index])

  the state is bordered by <a href="https://en.wikipedia.org/wiki/rajasthan" title="rajasthan">rajasthan</a> to the west, <a href="https://en.wikipedia.org/wiki/haryana" title="haryana">haryana</a>, <a href="https://en.wikipedia.org/wiki/himachal_pradesh" title="himachal pradesh">himachal pradesh</a> and <a href="https://en.wikipedia.org/wiki/delhi" title="delhi">delhi</a> to the northwest, <a href="https://en.wikipedia.org/wiki/uttarakhand" title="uttarakhand">uttarakhand</a> and an international border with <a href="https://en.wikipedia.org/wiki/nepal" title="nepal">nepal</a> to the north, <a href="https://en.wikipedia.org/wiki/bihar" title="bihar">bihar</a> to the east, <a href="https://en.wikipedia.org/wiki/madhya_pradesh" title="madhya pradesh">madhya pradesh</a> to the south, and touches the states of <a href="https://en.wikipedia.org/wiki/jharkhand" title="jharkhand">jharkhand</a> and <a href="https://en.wikipedia.org/wiki/chhattisgarh" title="chhattisgarh">chhattisgarh</a> to t

find the size of the index file

In [ ]:
import os

# Set the path of the file
file_path_ofEuclidiean = '/content/index_euclidDistance.bin'
file_path_ofInnerProduct = '/content/index_innerProduct.bin'

# Get the size of the file in bytes
file_size_forEculidian = os.path.getsize(file_path_ofEuclidiean)//1024
file_size_forInnerProduct = os.path.getsize(file_path_ofInnerProduct)//1024

# Print the size of the file
print("File size for eculidean index:", file_size_forEculidian , "KB")
print("File size for inner product index:", file_size_forInnerProduct , "KB")
#this is the size of index after adding embedding vector in index

File size for eculidean index: 936 KB
File size for inner product index: 468 KB
